<a href="https://colab.research.google.com/github/alikilic1631/aaaa/blob/main/DrugNet_Main_Document.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import joblib
import numpy as np
import pandas as pd
import sklearn
from sklearn import mixture
from keras.engine import Layer
import os
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import clip_by_global_norm
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.initializers import lecun_normal
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.losses import binary_crossentropy,MAE,MSE,CategoricalCrossentropy
from tensorflow.keras import layers as lyrs
from tensorflow.keras.layers import *
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Concatenate as concat
import tensorflow.keras.backend as back
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.math import reduce_logsumexp, reduce_mean, reduce_sum
from tensorflow.keras.backend import clip
from tensorflow.linalg import tensor_diag
from tensorflow.nn import softplus
from collections import Counter
import sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from time import time
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.regularizers import l2

# **Importing Preprocessed Data**

In [ ]:
selfies_data_path=str(input("Insert selfies_data_path: "))
x_train = np.load(selfies_data_path,allow_pickle=True)

# **Building Parts of MAE**

In [ ]:
latent_dim=84
mol_input_shape=(128)
mol_feat_num=84

In [ ]:
def mish(x):
  return x*back.tanh(back.softplus(x))

from keras.utils.generic_utils import get_custom_objects
get_custom_objects().update({'mish': Activation(mish)})

In [ ]:
def build_MAE(latent_dim,mol_input_shape,mol_feat_num):
  mol_input=Input(shape=(mol_input_shape))
  lyr=Embedding(mol_feat_num+1,mol_feat_num,input_length=mol_input_shape,mask_zero=True)(mol_input)

  lyr=Bidirectional(GRU(240,return_sequences=True),merge_mode='concat')(lyr)
  lyr=LayerNormalization()(lyr)

  lyr=Bidirectional(GRU(240,return_sequences=True),merge_mode='concat')(lyr)
  lyr=LayerNormalization()(lyr)

  #--------------------latent---------------------------------

  lyr=GlobalMaxPool1D()(lyr)
  latent_vector=Dense(latent_dim,activation="linear")(lyr)

  mae_encoder=Model(mol_input,latent_vector,name="mae_encoder")
  mae_encoder.summary()

  #-------------------decoder-------------------------------

  input=Input(shape=(latent_dim,))
  lyr=RepeatVector(mol_input_shape)(input)

  lyr=Reshape((mol_input_shape,latent_dim))(lyr)

  lyr=GRU(240*2,return_sequences=True,kernel_regularizer=l2(0.01))(lyr)
  lyr=LayerNormalization()(lyr)

  lyr=GRU(240*2,return_sequences=True,kernel_regularizer=l2(0.01))(lyr)
  lyr=LayerNormalization()(lyr)

  output=TimeDistributed(Dense(mol_feat_num+1, activation='softmax'))(lyr)
  mae_decoder=Model(input,output,name="mae_decoder")

  mae_decoder.summary()
  return mae_encoder,mae_decoder

In [ ]:
mae_encoder,mae_decoder=build_MAE(latent_dim,mol_input_shape,mol_feat_num)

Model: "rae_encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 128, 84)           7140      
_________________________________________________________________
bidirectional (Bidirectional (None, 128, 480)          469440    
_________________________________________________________________
layer_normalization (LayerNo (None, 128, 480)          960       
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128, 480)          1039680   
_________________________________________________________________
layer_normalization_1 (Layer (None, 128, 480)          960       
_________________________________________________________________
global_max_pooling1d (Global (None, 480)               

# **Regularized Autoencoder with L2 Regularization**






In [ ]:
mae_recon_loss=tf.keras.losses.SparseCategoricalCrossentropy()

def mae_reg_loss(latent_sample):
  return reduce_mean(back.square(latent_sample))/4

In [ ]:
class MAE(Model):
  def __init__(self,mae_encoder,mae_decoder,latent_dim):

    super(MAE,self).__init__()

    self.mae_encoder=mae_encoder
    self.mae_decoder=mae_decoder

    self.latent_dim=latent_dim

  def compile(self,mae_optimizer,
              mae_recon_loss,mae_reg_loss):
    super().compile()

    self.mae_optimizer=mae_optimizer

    self.mae_recon_loss=mae_recon_loss
    self.mae_reg_loss=mae_reg_loss

  def train_step(self,x):

    batch_size=back.shape(x)[0]

    with tf.GradientTape() as tape:
      latent_sample=self.mae_encoder(x,training=True)
      reconstructed_x=self.mae_decoder(latent_sample,training=True)

      mae_recon_loss=128*self.mae_recon_loss(x,reconstructed_x)
      mae_reg_loss=self.mae_reg_loss(latent_sample)

      mae_loss=mae_recon_loss+mae_reg_loss

    mae_gradient=tape.gradient(mae_loss,(self.mae_encoder.trainable_variables+self.mae_decoder.trainable_variables))
    self.mae_optimizer.apply_gradients(zip(mae_gradient,(self.mae_encoder.trainable_variables+self.mae_decoder.trainable_variables)))

    return {
        "MAE_Reconstruction_Loss": mae_recon_loss,
        "MAE_Reg_loss": mae_reg_loss
            }

In [ ]:
clipping_val=0.5
MAE_l2=MAE(mae_encoder,mae_decoder,latent_dim)

mae_optimizer=Adam(learning_rate=0.0005,
                    beta_1=0.9,
                    beta_2=0.99,
                   clipvalue=clipping_val)

MAE_l2.compile(mae_optimizer,mae_recon_loss,mae_reg_loss)

In [ ]:
encoder_path=str(input("Insert encoder_path: "))
decoder_path=str(input("Insert decoder_path: "))

In [ ]:
MAE_l2.mae_encoder.load_weights(encoder_path)
MAE_l2.mae_decoder.load_weights(decoder_path)

# **XPDE Training**

In [ ]:
bgmm_sampler=mixture.BayesianGaussianMixture(n_components=12,covariance_type='full',verbose=4,verbose_interval=5,warm_start=True)

In [ ]:
bgmm_sampler_path=str(input("Insert bgmm_sampler_path: "))
bgmm_sampler = joblib.load(bgmm_sampler_path)

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2021-01-28 19:05:43--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M  96.4MB/s    in 0.8s    

2021-01-28 19:05:44 (96.4 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37

In [ ]:
!pip install selfies

In [ ]:
!pip install SmilesPE

In [ ]:
from SmilesPE.pretokenizer import atomwise_tokenizer

In [ ]:
import selfies

In [ ]:
import networkx as nx
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem
from rdkit.Chem.Descriptors import qed, MolLogP
from rdkit.Chem import RDConfig
import os
import sys
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
import sascorer

In [ ]:
labels=['','[C]',
 '[#C]',
 '[=C]',
 '[c]',
 '[=c]',
 '[-c]',
 '[N]',
 '[#N]',
 '[=N]',
 '[-n]',
 '[n]',
 '[N+expl]',
 '[=N+expl]',
 '[=N-expl]',
 '[NHexpl]',
 '[=NH2+expl]',
 '[O]',
 '[=O]',
 '[o]',
 '[Oexpl]',
 '[O-expl]',
 '[F]',
 '[P]',
 '[S]',
 '[=S]',
 '[s]',
 '[Cl]',
 '[Cl-expl]',
 '[Br]',
 '[Br-expl]',
 '[I]',
 '[I-expl]',
 '[nHexpl]',
 '[Hexpl]',
 '[epsilon]',
 '[Branch1_1]',
 '[Branch1_2]',
 '[Branch1_3]',
 '[Branch2_1]',
 '[Branch2_2]',
 '[Branch2_3]',
 '[Branch3_3]',
 '[Branch3_1]',
 '[Branch3_2]',
 '[Ring1]',
 '[Expl-Ring1]',
 '[Expl=Ring1]',
 '[Ring2]',
 '[Expl-Ring2]',
 '[Expl=Ring2]',
 '[Ring3]',
 '.',
 '[C@@Hexpl]',
 '[C@Hexpl]',
 '[C@expl]',
 '[C@@expl]',
 '[/C]',
 '[\\C]',
 '[/S]',
 '[Expl\\Ring1]',
 '[/N]',
 '[\\N]',
 '[/C@Hexpl]',
 '[H+expl]',
 '[S-expl]',
 '[Zn+2expl]',
 '[Siexpl]',
 '[Na+expl]',
 '[\\S]',
 '[/O]',
 '[P@expl]',
 '[Expl/Ring2]',
 '[N-expl]',
 '[B]',
 '[\\C@@Hexpl]',
 '[/C@@Hexpl]',
 '[=S+expl]',
 '[Cexpl]',
 '[=Cexpl]',
 '[/Br]',
 '[/Cexpl]',
 '[/Cl]',
 '[Znexpl]',
 '[P@@expl]']

In [ ]:
mol_dict = dict(zip(range(0,85),labels))
mol_dict_2= dict(zip(labels,range(0,85)))

# **Deep ADTP**#

In [ ]:
def residual_block(lyr,filter_size,kernel_size,dilation=1,padding="same"):
  reslyr=lyr
  lyr=Conv1D(filter_size,kernel_size,activation="mish",padding=padding,dilation_rate=dilation)(lyr)
  lyr=BatchNormalization()(lyr)
  lyr=lyrs.Add()([lyr,reslyr])
  return lyr

In [ ]:
drug_input_shape=(128)
target_input_shape=(1500)
#_________________Target_________________

target_input=Input(shape=target_input_shape)
target_embedding=Embedding(21,170,mask_zero=True)(target_input)
tlyr=residual_block(target_embedding,170,6,1,"causal")

tlyr=residual_block(tlyr,170,6,2,"causal")
tlyr=residual_block(tlyr,170,6,4,"causal")

tlyr=residual_block(tlyr,170,6,8,"causal")
tlyr=Conv1D(170,6,activation="mish",padding="causal",dilation_rate=16)(tlyr)
tlyr=BatchNormalization()(tlyr)

#_________________Drug_________________

drug_input=Input(shape=drug_input_shape)
drug_embedding=Embedding(85,170,mask_zero=True)(drug_input)
dlyr=residual_block(drug_embedding,170,3,1,"causal")

dlyr=residual_block(dlyr,170,3,2,"causal")
dlyr=residual_block(dlyr,170,3,4,"causal")

dlyr=residual_block(dlyr,170,3,8,"causal")
dlyr=Conv1D(170,3,activation="mish",padding="causal",dilation_rate=16)(dlyr)
dlyr=BatchNormalization()(dlyr)

#______________Predictor______________

att_dlyr=Attention()([dlyr, tlyr])
att_dlyr=GlobalMaxPool1D()(att_dlyr)

att_tlyr=Attention()([tlyr, dlyr])
att_tlyr=GlobalMaxPool1D()(att_tlyr)

lyr=concat()([att_dlyr,att_tlyr])

lyr=Dense(5*170,activation="mish")(lyr)
lyr=lyrs.Dropout(0.1)(lyr)

lyr=Dense(5*170,activation="mish")(lyr)
lyr=lyrs.Dropout(0.1)(lyr)

lyr=Dense(5*170,activation="mish")(lyr)
output=Dense(1,activation="linear")(lyr)

deep_adtp=Model([target_input,drug_input],output,name="Deep_ADTP")
deep_adtp.summary()

Model: "Deep_ADTP"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1500)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 128, 170)     14450       input_4[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1500, 170)    3570        input_3[0][0]                    
__________________________________________________________________________________________

In [ ]:
deep_adtp_weights_path=str(input("Insert deep_adtp_weights_path: "))
deep_adtp.load_weights(deep_adtp_weights_path)

# **AES Search**

In [ ]:
covid_3cl_proteiase=np.array([1,2,3,4,5,6,7,3,8,1,2,5,9,10,2,11,6,9,12,9,13,11,2,13,13,13,14,15,2,14,16,14,17,17,9,
                     9,18,11,8,4,19,9,20,11,13,1,10,17,6,14,15,8,15,18,10,17,14,14,20,4,5,1,15,19,15,3,14,
                     9,12,7,2,15,9,12,14,4,9,20,2,19,1,6,12,15,11,9,14,5,14,5,9,17,13,7,15,8,5,13,8,5,18,5,
                     3,9,4,20,12,8,2,12,13,3,1,9,14,7,11,18,15,2,1,8,1,2,9,18,12,11,7,6,4,8,15,3,13,20,5,2,
                     1,3,14,15,2,1,11,2,1,9,2,3,15,20,17,18,17,11,9,1,3,11,18,6,19,19,6,10,14,8,13,2,9,19,7,
                     2,13,17,14,10,2,15,3,18,2,8,3,9,17,4,12,13,7,12,7,7,2,13,17,13,13,20,13,9,15,9,14,7,16,
                     14,18,7,7,9,20,15,2,17,4,16,3,14,15,4,3,13,13,13,14,15,17,3,15,14,9,7,6,5,18,15,18,10,8,
                     14,13,12,17,19,9,17,20,14,2,8,14,1,7,12,13,2,20,7,9,14,17,6,11,7,1,14,5,10,14,14,12,15,2,
                     6,15,2,4,13,20,14,2,1,7,14,14,10,17,10,3,13,8,3,17,9,9,4,12,11,1,2,9,13,3,12,],dtype=np.float32)

In [ ]:
covid_3cl_proteiase_new=np.empty((1,1500))
for j in range(306):
  covid_3cl_proteiase_new[0][j]=covid_3cl_proteiase[j]
for i in range(306,1500):
  covid_3cl_proteiase_new[0][i]=0

In [ ]:
covid_3cl_proteiase=covid_3cl_proteiase_new

In [ ]:
def onehot_to_tokenized(onehot):
  tokenized=np.empty([1,128])
  for i in range(128):
    tokenized[0][i]=np.argmax(onehot[0][i])
  return tokenized

In [ ]:
def onehot_to_smiles(onehot):
  selfies_mol=""
  for i in range(128):
    selfies_mol+=mol_dict[np.argmax(onehot[0][i])]
  smiles=selfies.decoder(selfies_mol)

  return smiles

In [ ]:
#Penalized LogP fonksiyonunun kodu You, J., Liu, B., Ying, R., Pande, V., & Leskovec, J. (2019, February 25).
#Graph Convolutional Policy Network for Goal-Directed Molecular Graph Generation. arXiv.org. https://arxiv.org/abs/1806.02473. adlı kağıttan alınmıştır.
#Baştan tanımlanan ortalama ve standart sapma parametreleri ZINC 250k veri seti üzerinden hesaplanmıştır.
def plogp_calculate(mol):

    logP_mean = 2.4570953396190123
    logP_std = 1.434324401111988
    SA_mean = -3.0525811293166134
    SA_std = 0.8335207024513095
    cycle_mean = -0.0485696876403053
    cycle_std = 0.2860212110245455

    try:
        log_p = MolLogP(mol)
    except ValueError:
        return 0
    try:
        SA = -sascorer.calculateScore(mol)
    except ZeroDivisionError:
        return 0

    cycle_list = nx.cycle_basis(nx.Graph(
        Chem.rdmolops.GetAdjacencyMatrix(mol)))
    if len(cycle_list) == 0:
        cycle_length = 0
    else:
        cycle_length = max([len(j) for j in cycle_list])
    if cycle_length <= 6:
        cycle_length = 0
    else:
        cycle_length = cycle_length - 6
    cycle_score = -cycle_length

    normalized_log_p = (log_p - logP_mean) / logP_std
    normalized_SA = (SA - SA_mean) / SA_std
    normalized_cycle = (cycle_score - cycle_mean) / cycle_std

    return normalized_log_p + normalized_SA + normalized_cycle

In [ ]:
def fitness_func(solution,typ):

  #COVID 3CL Proteiase score
  if typ="3CL":
    global covid_3cl_proteiase
    solution_mol=onehot_to_tokenized(mae_decoder.predict(solution))
    BindAff=deep_adtp.predict([covid_3cl_proteiase,solution_mol])

    return -BindAff

  #QED score
  elif typ=="QED"
    solution_mol=mae_decoder.predict(solution)
    mol=onehot_to_smiles(solution_mol)
    try:
      mol=Chem.CanonSmiles(mol)
      mol=Chem.MolFromSmiles(mol)
      qed_score=qed(mol)
    except:
      qed_score=0
    return -qed_score

  #PlogP score
  elif typ=="PlogP":
    solution_mol=mae_decoder.predict(solution)
    mol=onehot_to_smiles(solution_mol)
    try:
      mol=Chem.CanonSmiles(mol)
      mol=Chem.MolFromSmiles(mol)
      Plogp_score=plogp_calculate(mol)
    except:
      Plogp_score=-10
    return -Plogp_score

In [ ]:
class host():
  def __init__(self,h_type,solution):
    #Viral RNA that represents possible solution. is an array in array for model purposes
    self.solution = solution
    #Hosts level of health i.e healthy,mild,severe or critical
    self.h_type = h_type
    self.age=0
    #Intensity initialization functionında farklı olarak tanımlanacam
    self.m_intensity=solution.shape[0]/10 #1,-1
    self.s_intensity=1/np.random.random()
    #Currently used intensity vector which is added to the solution in the mutation function
    self.intensity = None
    #Fitness value gotten from the fitness function
    self.fitness = None

In [ ]:
class AES_Search(host):
  def __init__(self,latent_dim,decay,a,y_exploit,H,population_size,recov_percent,infection_rates,trans_probs,bgmm_sampler,fitness_type="3CL",VSO=False):

    #Latent Dimension
    self.latent_dim=latent_dim

    #Infection rates is a dictionary. It contains the probabilities of healthy hosts being infected by infected hosts depending on which type of infected host they are
    self.infection_rates=infection_rates
    self.init_inf_rates=infection_rates

    #Transformation probabilities is a dictionary. It contains the probabilities for an infected healthy host to be turned into a mild or severe host depending on which type of infected host they've been infected by
    self.trans_probs=trans_probs

    #decay is a coefficient in the calculation of s_intensity
    self.decay = decay
    self.init_decay=decay

    #a is a coefficient in the calculation of m_intensity
    self.a = a
    #y_exploit is a coefficient in the calculation of m_intensity
    self.y_exploit = y_exploit
    self.init_y_exploit=y_exploit

    #H is the maximum number of healthy hosts that can be contacted by an infected host
    self.H = H
    self.init_H=H

    #Number of hosts
    self.population_size = population_size
    #Recov percent is a percentage used in recovery
    self.recov_percent = recov_percent
    self.init_recov_percent=recov_percent

    #The current critical host of the search. This is an instance of the host class
    self.crit = None
    #The index of the critical host in the hosts list
    self.crit_index = None

    #Hosts is a list which contains all population_size of hosts. Hosts have solution, h_type,...
    self.hosts = []
    #Infected and healthy hosts are dictionaries. These dictionaries contain the infected and healthy hosts of the hosts list as their keys and their indexes in the hosts list as thier values
    self.infected_hosts = {}
    self.healthy_hosts = {}

    #Random solution generator. Makes np.array random solutions
    self.bgmm_sampler=bgmm_sampler
    self.crit_age=0

    #Sum of healthy host fitness values for each epoch
    self.healthy_host_loss=0
    #Sum of mild and severe host fitness values for each epoch
    self.infected_host_loss=0

    self.infected_count=0
    self.healthy_count=0

    self.fitness_type=fitness_type
    self.VSO=VSO


  #Initialize is the first initialization of all random hosts.
  #Works! --> Bound işlemlerine bak, sayfa 5 altı
  def initialize(self):
    for i in range(self.population_size):
      rand_sol=self.bgmm_sampler.sample()[0]
      self.hosts.append(host('healthy',rand_sol))

  #Selection is where all hosts are given their fitness values. After this the one with the best fitness value is chosenn as the critical host
  #Works! --> Crit age şeylerine bak, bi de reset function
  def selection(self):
    #Crit age bak
    self.crit_age+=1

    for i in range(len(self.hosts)):

      if self.crit == None:
        self.hosts[i].h_type = "critical"
        self.hosts[i].fitness = fitness_func(self.hosts[i].solution,self.fitness_type)
        self.crit_index = i
        self.crit = self.hosts[i]

        #Crit age bak
        self.crit_age=0
      else:
        self.hosts[i].fitness = fitness_func(self.hosts[i].solution,self.fitness_type)
        if self.crit.fitness>self.hosts[i].fitness:
          self.hosts[i].h_type = "critical"
          self.hosts[self.crit_index].h_type = "severe"
          self.crit_index = i
          self.crit = self.hosts[i]

          #Crit age bak
          self.crit_age=0

          #Reset function bak
          self.reset()

      if self.hosts[i].h_type=="healthy":
        self.healthy_count+=1
        self.healthy_host_loss+=self.hosts[i].fitness
      elif self.hosts[i].h_type=="mild" or self.hosts[i].h_type=="severe":
        self.infected_count+=1
        self.infected_host_loss+=self.hosts[i].fitness


  #Mutation açıklamasını yazarsın
  #Works! --> Random functionlarını kontrol et
  def mutate(self):
    for i in range(len(self.hosts)):
      if self.hosts[i].h_type == "healthy":
        self.hosts[i].solution+=np.random.uniform(low=0,high=2,size=[1,self.latent_dim])*(self.crit.solution+self.hosts[i].solution)
        self.hosts[i].fitness = fitness_func(self.hosts[i].solution,self.fitness_type)

      elif self.hosts[i].h_type == "mild":
        self.hosts[i].intensity = self.a * self.hosts[i].m_intensity + self.y_exploit * np.random.random([1,self.latent_dim]) * (self.crit.solution - self.hosts[i].solution)
        self.hosts[i].solution += self.hosts[i].intensity
        self.hosts[i].fitness = fitness_func(self.hosts[i].solution,self.fitness_type)

      elif self.hosts[i].h_type == "severe":
        self.hosts[i].intensity = self.decay * self.hosts[i].s_intensity
        self.hosts[i].solution += self.hosts[i].solution * np.random.normal(size = [1,self.latent_dim],scale=self.hosts[i].intensity)
        self.hosts[i].fitness = fitness_func(self.hosts[i].solution,self.fitness_type)

  #Divides the hosts list into healthy and infected, more info on the variables at the top
  def make_lists(self):
    self.healthy_hosts = {}
    self.infected_hosts = {}
    for i in range(len(self.hosts)):
      if self.hosts[i].h_type == "healthy":
        self.healthy_hosts[self.hosts[i]] = i
      else:
        self.infected_hosts[self.hosts[i]] = i

  #Random functionlarına bak
  #Works!
  def infect(self):
    #Index variables:
    #i is the index for infected_fitness_sorted
    #j is the index for healthy_fitness_sorted

    #Infected fitness sorted is a list that only contains the infected hosts in an ascending order of their fitness values
    infected_fitness_sorted = [a for a in self.infected_hosts.keys()]
    infected_fitness_sorted.sort(key = lambda i: i.fitness)

    for i in range(len(infected_fitness_sorted)):
      #For this if statement we need to use the healthy hosts dictionary because the healthy fitness sorted list hasn't been made yet
      if len(list(self.healthy_hosts.keys())) >= self.H:
        #Healthy fitness sorted is a list that only contains the healthy hosts in an descending order of their fitness values
        healthy_fitness_sorted = [a for a in self.healthy_hosts.keys()]
        healthy_fitness_sorted.sort(key = lambda g: g.fitness,reverse = True)

        #Contacted hosts are the first H hosts of the healthy_fitness_sorted list which is sorted according to descending fitness value
        contacted_hosts = healthy_fitness_sorted[:self.H]

        for j in range(len(contacted_hosts)):
          #Infected is whether the healthy host has been infected or not
          infected = False
          #T is the h_type of the infecting host
          T = infected_fitness_sorted[i].h_type
          #Infection rate is determined by the h_type of the infecting host
          infection_rate = self.infection_rates[T]

          #Decides whether the healthy host will be infected or not
          if infection_rate >= np.random.random():
            infected = True

          #If infected, the healthy host turns into a mild or severe host
          if infected == True:
            #Transformation probabilities
            tp1, tp2 = self.trans_probs[T][0], self.trans_probs[T][1]

            #Num is the random number that decides whether the healthy host will turn into a mild or severe host
            num = np.random.random()

            if 0 <= num and num <= tp1:
              to_be_infected_type = "mild"

            else:
              to_be_infected_type = "severe"

            host_index = self.healthy_hosts[healthy_fitness_sorted[j]]

            if to_be_infected_type == "mild":
              healthy_fitness_sorted[j].h_type = "mild"

              for idx in range(self.latent_dim):
                if 0.5 >= np.random.random():
                  healthy_fitness_sorted[j].solution[0][idx] = infected_fitness_sorted[i].solution[0][idx]
                  healthy_fitness_sorted[j].fitness = fitness_func(healthy_fitness_sorted[j].solution)

            elif to_be_infected_type == "severe":
              healthy_fitness_sorted[j].h_type = "severe"
              healthy_fitness_sorted[j].solution = infected_fitness_sorted[i].solution
              healthy_fitness_sorted[j].fitness = fitness_func(healthy_fitness_sorted[j].solution)

            self.hosts[host_index] = healthy_fitness_sorted[j]
            self.make_lists()

  def recover(self):
    infected_fitness_sorted = [a for a in self.infected_hosts.keys()]
    infected_fitness_sorted.sort(key = lambda i: i.fitness,reverse=True)

    if len(infected_fitness_sorted) == self.population_size:
      #Rev num is the number of infected hosts that will recover
      rev_num = self.recov_percent*self.population_size
      recovery_hosts = infected_fitness_sorted[:int(rev_num)]

      for i in range(len(recovery_hosts)):
        host_index = self.infected_hosts[recovery_hosts[i]]
        recovery_hosts[i].solution = self.bgmm_sampler.sample()[0]
        recovery_hosts[i].fitness = fitness_func(recovery_hosts[i].solution)

        if recovery_hosts[i].h_type=="severe":
          recovery_hosts[i].h_type = "mild"

        elif recovery_hosts[i].h_type == "mild":
          recovery_hosts[i].h_type = "healthy"

        self.hosts[host_index] = recovery_hosts[i]

  def adaptation_function(self,crit_age):
    return 1/(1+np.e**(9-crit_age))

  def adaptation(self):
    if self.VSO==False
      p_coeff=self.adaptation_function(self.crit_age)

      self.recov_percent=min(p_coeff,0.75)
      self.y_exploit=p_coeff+1

      for i in self.infection_rates.keys():
        self.infection_rates[i]/=p_coeff+1

      self.H=(int(self.init_H-(p_coeff*self.init_H)))
      self.decay=min(int(self.init_decay+(p_coeff/10)),0.99)
    else:
      continue

  def reset(self):
    if self.VSO==False
      self.decay=self.init_decay
      self.H=self.init_H
      self.infection_rates=self.init_inf_rates
      self.recov_percent=self.init_recov_percent
      self.y_exploit=self.init_y_exploit
    else:
      continue

  def results(self):
    print(
        "\nCritical_Host_Loss:",-self.crit.fitness,
        "\nCrit-Age:",self.crit_age,
        "\nHealthy_Host_Loss:",-1*self.healthy_host_loss/self.healthy_count,
        "\nInfected_Host_Loss:",-1*self.infected_host_loss/self.infected_count,
    )
    self.infected_host_loss=0
    self.healthy_host_loss=0
    self.infected_count=0
    self.healthy_count=0

In [ ]:
AES_module=AES_Search(latent_dim,0.9,1,2,20,400,0.3,
                      {"mild":0.2,"severe":0.5,"critical":0.8},
                      {"mild":[0.7,0.3],"severe":[0.5,0.5],"critical":[0.3,0.7]},
                      bgmm_sampler,False)

# **Produce Potential Drugs**

In [ ]:
AES_module.initialize()
AES_module.selection()

In [ ]:
num_epoch=int(input("Enter Epoch:"))
for i in range(num_epoch):
  AES_module.mutate()
  AES_module.make_lists()
  AES_module.infect()
  AES_module.make_lists()
  AES_module.recover()
  AES_module.make_lists()
  AES_module.adaptation()
  AES_module.selection()
  AES_module.results()

In [ ]:
infected=list(AES_module.infected_hosts.keys())
lenght_count=0
for j in range(len(infected)):
  if infected[j].h_type=="severe" or infected[j].h_type=="critical":
    lenght_count+=1

generated_solutions_1=np.empty((1,lenght_count,latent_dim))
index_count=0

for i in range(len(infected)):
  if infected[i].h_type=="severe" or infected[i].h_type=="critical":
    generated_solutions[0][index_count]=infected[i].solution
    index_count+=1

In [ ]:
generated_mols_path=str(input("Insert Path: "))
np.save(generated_mols_path,generated_sols)